# SageMaker Pipelines Customer Churn Prediction

Amazon SageMaker Model Building Pipelines offers machine learning (ML) application developers and operations engineers the ability to orchestrate SageMaker jobs and author reproducible ML pipelines. It also enables them to deploy custom-build models for inference in real-time with low latency, run offline inferences with Batch Transform, and track lineage of artifacts. They can institute sound operational practices in deploying and monitoring production workflows, deploying model artifacts, and tracking artifact lineage through a simple interface, adhering to safety and best practice paradigms for ML application development.

The SageMaker Pipelines service supports a SageMaker Pipeline domain specific language (DSL), which is a declarative JSON specification. This DSL defines a directed acyclic graph (DAG) of pipeline parameters and SageMaker job steps. The SageMaker Python Software Developer Kit (SDK) streamlines the generation of the pipeline DSL using constructs that engineers and scientists are already familiar with.

## SageMaker Pipelines

SageMaker Pipelines supports the following activities, which are demonstrated in this notebook:

* Pipelines - A DAG of steps and conditions to orchestrate SageMaker jobs and resource creation.
* Processing job steps - A simplified, managed experience on SageMaker to run data processing workloads, such as feature engineering, data validation, model evaluation, and model interpretation.
* Training job steps - An iterative process that teaches a model to make predictions by presenting examples from a training dataset.
* Conditional execution steps - A step that provides conditional execution of branches in a pipeline.
* Register model steps - A step that creates a model package resource in the Model Registry that can be used to create deployable models in Amazon SageMaker.
* Create model steps - A step that creates a model for use in transform steps or later publication as an endpoint.
* Parametrized Pipeline executions - Enables variation in pipeline executions according to specified parameters.

## Notebook Overview

This notebook shows how to:

* Define a set of Pipeline parameters that can be used to parametrize a SageMaker Pipeline.
* Define a Processing step that extracts data from feature store to create the train, validation and test data sets.
* Define a Training step that trains a model on the preprocessed train data set.
* Define a Processing step that evaluates the trained model's performance on the test dataset.
* Define a Create Model step that creates a model from the model artifacts used in training.
* Define a Conditional step that measures a condition based on output from prior steps and conditionally executes other steps.
* Define a Register Model step that creates a model package from the estimator and model artifacts used to train the model.
* Define a Processing step that delopy a real-time endpoint from the trained model.
* Define and create a Pipeline definition in a DAG, with the defined parameters and steps.
* Start a Pipeline execution and wait for execution to complete.
* Download the model evaluation report from the S3 bucket for examination.
* Clean up all artifacts created in this sample code.

## A SageMaker Pipeline

The pipeline that you create follows a typical machine learning (ML) application pattern of preprocessing, training, evaluation, model creation, model registration, batch transform and endpoint deployment.

<div align="center">
    <img width=800 src="images/mlworkflow.png"><figcaption>ML workflow with SageMaker Pipeline</figcaption>
</div>


## 1. Preparation

**Required IAM policies**
- IAMFullAccess
- AmazonSQSFullAccess

### 1.1 Serverless services preparation

In this session, we create resources including `SQS`, `Lambda Function`, `Step Functions` and `AWS Glue job` for data preprocessing. We use  [AWS Cloud Development Kit (AWS CDK)](https://docs.aws.amazon.com/cdk/v1/guide/home.html) to create these resources by following the steps beflow:

- Step-1, clone this repo to local or [cloud9](https://aws.amazon.com/cloud9/)
- Step-2, [install and set CDK environment](https://docs.aws.amazon.com/cdk/v1/guide/getting_started.html), we use cdk1.170.0 in this repo
- Step-3, run `cd {project_root}/sagemaker-pipeline/cfn`
- Step-4, run `cdk synth`
- Step-5, run `cdk deploy --CfnStack --parameters ExecutionRoleArn={the role name of your current notebook instance}`. you can get the following result once deployment succeeded.
<div align="center">
    <img width=800 src="./images/cdk_deploy.png"></img>
</div>

Copy the `CfnStack.SqsURL` and will use it in Session-4.

### 1.2 Python libraries import

**Let's start by specifying:**

- The S3 bucket and prefix that you want to use for training and model data.  This should be within the same region as the Notebook Instance, training, and hosting.
- The IAM role arn used to give training and hosting access to your data. See the documentation for how to create these.  Note, if more than one role is required for notebook instances, training, and/or hosting, please replace the boto regexp with a the appropriate full IAM role arn string(s).

In [1]:
import json
import os

import boto3
import sagemaker
import sagemaker.session
from sagemaker import get_execution_role

import sagemaker
from sagemaker.workflow.callback_step import (
    CallbackOutput,
    CallbackOutputTypeEnum,
    CallbackStep,
)
from sagemaker.workflow.lambda_step import (
    LambdaOutput,
    LambdaOutputTypeEnum,
    LambdaStep,
)
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
)
from sagemaker.workflow.steps import (
    ProcessingStep,
    Step,
    TrainingStep,
    CreateModelStep
)

from sagemaker.workflow.condition_step import ConditionStep
from sagemaker.workflow.conditions import ConditionGreaterThanOrEqualTo
from sagemaker.workflow.functions import Join, JsonGet
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.step_collections import RegisterModel
from sagemaker.workflow.properties import PropertyFile

from sagemaker.inputs import TrainingInput, CreateModelInput
from sagemaker.model_metrics import MetricsSource, ModelMetrics
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import (
    ProcessingInput,
    ProcessingOutput,
    Processor,
    ScriptProcessor,
)

from sagemaker.xgboost.estimator import XGBoost
from sagemaker.model import Model

import uuid

In [2]:
role = get_execution_role()

region = boto3.Session().region_name

sagemaker_session = sagemaker.Session()
account = sagemaker_session.boto_session.client("sts").get_caller_identity()["Account"]

bucket=sagemaker.Session().default_bucket()
prefix = 'sagemaker/DEMO-xgboost-customer-churn-connect'
base_job_prefix = 'Demo-xgboost-churn-connect'

In [3]:
bucket

'sagemaker-us-east-1-822507008821'

upload data to s3 input location

In [4]:
s3_client = boto3.client("s3")
s3_client.upload_file('../data/churn_processed.csv', bucket, prefix+'/input/churn_processed.csv')
inputDir = f"s3://{bucket}/{prefix}/input/"

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


## 2. Define Parameters to Parametrize Pipeline Execution

Define Pipeline parameters that you can use to parametrize the pipeline. Parameters enable custom pipeline executions and schedules without having to modify the Pipeline definition.

The supported parameter types include:

* `ParameterString` - represents a `str` Python type
* `ParameterInteger` - represents an `int` Python type
* `ParameterFloat` - represents a `float` Python type

These parameters support providing a default value, which can be overridden on pipeline execution. The default value specified should be an instance of the type of the parameter.

The parameters defined in this workflow include:

* `processing_instance_type` - The `ml.*` instance type of the processing job.
* `processing_instance_count` - The instance count of the processing job.
* `training_instance_type` - The `ml.*` instance type of the training job.
* `train_instance_count` - The instance count of the training job.
* `model_approval_status` - What approval status to register the trained model with for CI/CD purposes ( "PendingManualApproval" is the default).
* `model_output` - The S3 bucket URI location of the model output path

**You must have run the previous sequential notebooks to retrieve variables using the StoreMagic command.**

In [5]:
# parameters for pipeline execution

processing_instance_count = ParameterInteger(
    name="ProcessingInstanceCount", default_value=1
)
processing_instance_type = ParameterString(
    name="ProcessingInstanceType", default_value="ml.m5.xlarge"
)

train_instance_count = ParameterInteger(
    name="TrainingInstanceCount",
    default_value=1,
)
train_instance_type = ParameterString(
    name="TrainingInstance",
    default_value="ml.m5.xlarge",
)

model_approval_status = ParameterString(
    name="ModelApprovalStatus",
    default_value="Approved",
    enum_values=[
        "PendingManualApproval",
        "Approved",
    ],
)
model_output = ParameterString(
    name="ModelOutputUrl",
    default_value=f"s3://{bucket}/{prefix}/model",
)

## 3. Pipeline Steps
### 3.1 Callback Step
First, we'll configure the callback step.

The callback step will accept the following inputs:
- S3 location for train/validation/test dataset
- S3 location for the raw data
- arn of the Step Functions for preprocessing data

The callback step will return the following outputs:

- S3 location of processed data to be used for model training, including train data, validation data and test data.

In [6]:
queue_url = "https://sqs.us-east-1.amazonaws.com/822507008821/CfnStack-pipelinecallbacksglueprep0F0C1313-GKfQjQWTy24d" # Use CfnStack.SqsURL generated in cdk deployment.

In [7]:
from sagemaker.workflow.callback_step import CallbackStep,CallbackOutput,CallbackOutputTypeEnum

train_uri = CallbackOutput(output_name="trainUri", output_type=CallbackOutputTypeEnum.String)
val_uri = CallbackOutput(output_name="valUri", output_type=CallbackOutputTypeEnum.String)
test_uri = CallbackOutput(output_name="testUri", output_type=CallbackOutputTypeEnum.String)

step_callback_data = CallbackStep(
                    name="GluePrepCallbackStep",
                    sqs_queue_url=queue_url,
                    inputs={
                        "trainUri": f"s3://{bucket}/{prefix}/processed/train/train.csv",
                        "valUri": f"s3://{bucket}/{prefix}/processed/validation/validation.csv",
                        "testUri": f"s3://{bucket}/{prefix}/processed/test/test.csv",
                        "inputDir": inputDir
                    },
                    outputs=[
                        train_uri,
                        val_uri,
                        test_uri
                    ]
                )

### 3.2 Define a TrainStep

#### Define a Training Step to Train a Model

In this section, use Amazon SageMaker's [XGBoost Algorithm](https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost.html) to train on this dataset. Configure an Estimator for the XGBoost algorithm and the input dataset. A typical training script loads data from the input channels, configures training with hyperparameters, trains a model, and saves a model to `model_output` so that it can be hosted later. 

The model path where the models from training will be saved is also specified.

Note the `training_instance_type` parameter may be used in multiple places in the pipeline. In this case, the `training_instance_type` and `train_instance_count` is passed into the estimator.

In [8]:
# training step for generating model artifacts
image_uri = sagemaker.image_uris.retrieve(
    framework="xgboost",
    region=region,
    version="1.0-1",
    py_version="py3",
)
xgb_train = sagemaker.estimator.Estimator(
    image_uri=image_uri,
    instance_type=train_instance_type,
    instance_count=train_instance_count,
    output_path=model_output,
    base_job_name=f"{base_job_prefix}-train",
    sagemaker_session=sagemaker_session,
    role=role,
)

# Set some hyper parameters
# https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost_hyperparameters.html
xgb_train.set_hyperparameters(
    max_depth=5,
    eta=0.2,
    gamma=4,
    min_child_weight=6,
    subsample=0.8,
    silent=0,
    objective="binary:logistic",
    num_round=100,
    eval_metric='auc'
)


Finally, use the estimator instance to construct a `TrainingStep` as well as the `properties` of the prior `ProcessingStep` used as input in the `TrainingStep` inputs and the code that's executed when the pipeline invokes the pipeline execution. This is similar to an estimator's `fit` method in the Python SDK.

Pass in the `S3Uri` of the `"train"` output channel to the `TrainingStep`. Also, use the other `"validation"` output channel for model evaluation in the pipeline. The `properties` attribute of a Pipeline step matches the object model of the corresponding response of a describe call. These properties can be referenced as placeholder values and are resolved at runtime. For example, the `ProcessingStep` `properties` attribute matches the object model of the [DescribeProcessingJob](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_DescribeProcessingJob.html) response object.

In [9]:
step_train = TrainingStep(
    name="TrainModel",
    estimator=xgb_train,
    inputs={
        "train": TrainingInput(
            s3_data=train_uri,
            content_type="text/csv",
        ),
        "validation": TrainingInput(
            s3_data=val_uri,
            content_type="text/csv",
        ),
    },

)

### 3.3 Define a Model Evaluation Step to Evaluate the Trained Model

First, develop an evaluation script that is specified in a Processing step that performs the model evaluation.

After pipeline execution, you can examine the resulting `evaluation.json` for analysis.

The evaluation script uses `xgboost` to do the following:

* Load the model.
* Read the test data.
* Issue predictions against the test data.
* Build a classification report, including mae, mse, rmse and r2 metrics.
* Save the evaluation report to the evaluation directory.

Next, create an instance of a `ScriptProcessor` processor and use it in the `ProcessingStep`.

Note the `processing_instance_type` parameter passed into the processor.

In [10]:
# processing step for evaluation
script_eval = ScriptProcessor(
    image_uri=image_uri,
    command=["python3"],
    instance_type=processing_instance_type,
    instance_count=1,
    base_job_name=f"{base_job_prefix}/script-eval",
    sagemaker_session=sagemaker_session,
    role=role,
)


Use the processor instance to construct a `ProcessingStep`, along with the input and output channels and the code that will be executed when the pipeline invokes pipeline execution. This is similar to a processor instance's `run` method in the Python SDK.

Specifically, the `S3ModelArtifacts` from the `step_train` `properties` and the `S3Uri` of the `"test"` output channel of the `step_process` `properties` are passed into the inputs. The `TrainingStep` and `ProcessingStep` `properties` attribute matches the object model of the [DescribeTrainingJob](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_DescribeTrainingJob.html) and  [DescribeProcessingJob](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_DescribeProcessingJob.html) response objects, respectively.

In [11]:
evaluation_report = PropertyFile(
    name="EvaluationReport",
    output_name="evaluation",
    path="evaluation.json",
)
eval_output = f"s3://{bucket}/{prefix}/Evaluation/output/"
step_eval = ProcessingStep(
    name="EvaluateModel",
    processor=script_eval,
    inputs=[
        ProcessingInput(
            source=step_train.properties.ModelArtifacts.S3ModelArtifacts,
            destination="/opt/ml/processing/model",
        ),
        ProcessingInput(
            source=f"s3://{bucket}/{prefix}/processed/test/test.csv",
            destination="/opt/ml/processing/test",
        ),
    ],
    outputs=[
        ProcessingOutput(
            output_name="evaluation", 
            source="/opt/ml/processing/evaluation",
            destination=eval_output
        ),
    ],
    code="./code/evaluation.py",
    property_files=[evaluation_report],

)

### 3.4 Define a Create Model Step to Create a Model

In order to perform batch transformation using the example model, create a SageMaker model. 

Specifically, pass in the `S3ModelArtifacts` from the `TrainingStep`, `step_train` properties. The `TrainingStep` `properties` attribute matches the object model of the [DescribeTrainingJob](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_DescribeTrainingJob.html) response object.

In [12]:
model = Model(
    image_uri=image_uri,
    model_data=step_train.properties.ModelArtifacts.S3ModelArtifacts,
    sagemaker_session=sagemaker_session,
    role=role,
)

inputs = CreateModelInput(
    instance_type="ml.m5.xlarge",
)
step_create_model = CreateModelStep(
    name="CustomerChurnCreateModel",
    model=model,
    inputs=inputs,
)

### 3.5 Define a Register Model Step to Create a Model Package

Use the estimator instance specified in the training step to construct an instance of `RegisterModel`. The result of executing `RegisterModel` in a pipeline is a model package. A model package is a reusable model artifacts abstraction that packages all ingredients required for inference. Primarily, it consists of an inference specification that defines the inference image to use along with an optional model weights location.

A model package group is a collection of model packages. A model package group can be created for a specific ML business problem, and new versions of the model packages can be added to it. Typically, customers are expected to create a ModelPackageGroup for a SageMaker pipeline so that model package versions can be added to the group for every SageMaker Pipeline run.

The construction of `RegisterModel` is similar to an estimator instance's `register` method in the Python SDK.

Specifically, pass in the `S3ModelArtifacts` from the `TrainingStep`, `step_train` properties. The `TrainingStep` `properties` attribute matches the object model of the [DescribeTrainingJob](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_DescribeTrainingJob.html) response object.

Note that the specific model package group name provided in this notebook can be used in the model registry and CI/CD work with SageMaker Projects.

In [13]:
# register model step that will be conditionally executed
model_metrics = ModelMetrics(
    model_statistics=MetricsSource(
        s3_uri=eval_output,
        content_type="application/json",
    )
)
model_package_group_name = "CustomerChurnModelPackage"
step_register = RegisterModel(
    name="RegisterModel",
    estimator=xgb_train,
    model_data=step_train.properties.ModelArtifacts.S3ModelArtifacts,
    content_types=["text/csv"],
    response_types=["text/csv"],
    inference_instances=["ml.t2.medium", "ml.t2.large", "ml.m5.large"],
    transform_instances=["ml.m5.large"],
    model_package_group_name=model_package_group_name,
    approval_status=model_approval_status,
    model_metrics=model_metrics,
)

### 3.6 Deploy a model

In [14]:
deploy_model_script_uri = f"s3://{bucket}/{prefix}/code/deploy_model.py"
deploy_model_instance_type = "ml.m4.xlarge"
endpoint_name = "demo-customer-churn-xgboost"

s3_client.upload_file(
    Filename="code/deploy_model.py", Bucket=bucket, Key=f"{prefix}/code/deploy_model.py"
)

deploy_model_processor = SKLearnProcessor(
    framework_version="0.23-1",
    role=role,
    instance_type="ml.t3.medium",
    instance_count=1,
    base_job_name="customer-churn-deploy-model",
    sagemaker_session=sagemaker_session,
)

deploy_step = ProcessingStep(
    name="DeployModel",
    processor=deploy_model_processor,
    job_arguments=[
        "--model-name",
        step_create_model.properties.ModelName,
        "--region",
        region,
        "--endpoint-instance-type",
        deploy_model_instance_type,
        "--endpoint-name",
        endpoint_name,
    ],
    code=deploy_model_script_uri,
)

### 3.7 Define a Condition Step to Check Accuracy and Conditionally Create a Model and Run a Batch Transformation and Register a Model in the Model Registry

In this step, the model is registered only if the accuracy of the model, as determined by the evaluation step `step_eval`, exceeded a specified value. A `ConditionStep` enables pipelines to support conditional execution in the pipeline DAG based on the conditions of the step properties. 

In the following section, you:

* Define a `ConditionLessThanOrEqualTo` on the accuracy value found in the output of the evaluation step, `step_eval`.
* Use the condition in the list of conditions in a `ConditionStep`.
* Pass the `CreateModelStep` and `TransformStep` steps, and the `RegisterModel` step collection into the `if_steps` of the `ConditionStep`, which are only executed, if the condition evaluates to `True`.

In [15]:
# condition step for evaluating model quality and branching execution
cond_lte = ConditionGreaterThanOrEqualTo(
    left=JsonGet(
        step_name=step_eval.name,
        property_file=evaluation_report,
        json_path="binary_classification_metrics.accuracy.value",
    ),
    right=0.95,
)
step_cond = ConditionStep(
    name="CheckEvaluation",
    conditions=[cond_lte],
    if_steps=[step_create_model, step_register, 
              deploy_step],
    else_steps=[],
)

### 3.8 Define a Pipeline of Parameters, Steps, and Conditions

In this section, combine the steps into a Pipeline so it can be executed.

A pipeline requires a `name`, `parameters`, and `steps`. Names must be unique within an `(account, region)` pair.

Note:

* All of the parameters used in the definitions must be present.
* Steps passed into the pipeline do not have to be listed in the order of execution. The SageMaker Pipeline service resolves the _data dependency_ DAG as steps for the execution to complete.
* Steps must be unique to across the pipeline step list and all condition step if/else lists.

In [16]:
# pipeline instance
pipeline = Pipeline(
    name="Demo-customer-churn-pipeline",
    parameters=[
        processing_instance_count,
        processing_instance_type,
        train_instance_count,
        train_instance_type,
        model_approval_status,
        model_output
    ],
    steps=[step_callback_data,
           step_train,
           step_eval,
           step_cond],
    sagemaker_session=sagemaker_session,
)

### 3.9 Submit the pipeline to SageMaker and start execution

Submit the pipeline definition to the Pipeline service. The role passed in will be used by the Pipeline service to create all the jobs defined in the steps.

In [17]:
pipeline.upsert(role_arn=role)

No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config
No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config


{'PipelineArn': 'arn:aws:sagemaker:us-east-1:822507008821:pipeline/demo-customer-churn-pipeline',
 'ResponseMetadata': {'RequestId': '37afb3c6-88d9-4438-b997-faa631211f80',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '37afb3c6-88d9-4438-b997-faa631211f80',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '96',
   'date': 'Fri, 02 Sep 2022 08:23:27 GMT'},
  'RetryAttempts': 0}}

#### Pipeline Operations: Examining and Waiting for Pipeline Execution

Describe the pipeline execution.

In [18]:
execution = pipeline.start()

In [19]:
execution.describe()

{'PipelineArn': 'arn:aws:sagemaker:us-east-1:822507008821:pipeline/demo-customer-churn-pipeline',
 'PipelineExecutionArn': 'arn:aws:sagemaker:us-east-1:822507008821:pipeline/demo-customer-churn-pipeline/execution/z0q7hrouxfcu',
 'PipelineExecutionDisplayName': 'execution-1662107007900',
 'PipelineExecutionStatus': 'Executing',
 'CreationTime': datetime.datetime(2022, 9, 2, 8, 23, 27, 828000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2022, 9, 2, 8, 23, 27, 828000, tzinfo=tzlocal()),
 'CreatedBy': {},
 'LastModifiedBy': {},
 'ResponseMetadata': {'RequestId': '73da386b-3548-41b6-8047-f9c7e62264d6',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '73da386b-3548-41b6-8047-f9c7e62264d6',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '421',
   'date': 'Fri, 02 Sep 2022 08:23:28 GMT'},
  'RetryAttempts': 0}}

In [ ]:
# Wait for the execution to complete.
# execution.wait()

### 3.10 Examining the Evalution

Examine the resulting model evaluation after the pipeline completes. Download the resulting `evaluation.json` file from S3 and print the report.

In [ ]:
from pprint import pprint


evaluation_json = sagemaker.s3.S3Downloader.read_file("{}evaluation.json".format(
    step_eval.arguments["ProcessingOutputConfig"]["Outputs"][0]["S3Output"]["S3Uri"]
))
pprint(json.loads(evaluation_json))

#### Lineage

Review the lineage of the artifacts generated by the pipeline.

In [ ]:
import time
from sagemaker.lineage.visualizer import LineageTableVisualizer


viz = LineageTableVisualizer(sagemaker.session.Session())
for execution_step in reversed(execution.list_steps()):
    print(execution_step)
    display(viz.show(pipeline_execution_step=execution_step))
    time.sleep(5)

## 4. Clean up

In [ ]:
# Delete pipeline
pipeline.delete()

sfn_client = boto3.client('sagemaker')
sfn_client.delete_endpoint(EndpointName=endpoint_name)

# Delete artefacts
!aws s3 rm s3://$bucket/$prefix

You can go to [CloudFormation console](https://us-east-1.console.aws.amazon.com/cloudformation/home?region=us-east-1#/stacks?filteringStatus=active&filteringText=&viewNested=true&hideStacks=false) to delete the stack to remove all serverless resources such sqs, lambda and glue.

# GitHub Resource
This demo is available on GitHub: https://github.com/aws-samples/real-time-churn-prediction-with-amazon-connect-and-amazon-sagemaker